In [1]:
import argparse
import cv2 as cv
import numpy as np
from datetime import datetime

In [ ]:


# ==== CONFIG (edit these if needed) ====
CAMERA_INDEX = 0       # 0 = default camera
CLOAK_COLOR = "red"    # choose: "red", "blue", "green"
SHOW_MASKS = False     # True = show mask windows
# ======================================

def get_color_ranges(color: str):
    """Return HSV ranges for cloak color."""
    color = color.lower()
    if color == "red":
        return [
            (np.array([0, 120, 70]),   np.array([10, 255, 255])),
            (np.array([170, 120, 70]), np.array([180, 255, 255])),
        ]
    elif color == "blue":
        return [(np.array([94, 80, 2]), np.array([126, 255, 255]))]
    elif color == "green":
        return [(np.array([35, 80, 2]), np.array([85, 255, 255]))]
    else:
        raise ValueError("Supported colors: red, blue, green")

def build_mask(hsv, ranges):
    """Create a mask for the cloak color and clean it with morphology."""
    mask_total = None
    for lower, upper in ranges:
        mask = cv.inRange(hsv, lower, upper)
        mask_total = mask if mask_total is None else (mask_total | mask)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv.morphologyEx(mask_total, cv.MORPH_OPEN, kernel, iterations=2)
    mask = cv.dilate(mask, np.ones((7, 7), np.uint8), iterations=1)
    return mask

def capture_background(cap, frames=60):
    """Capture a clean background (step out of frame first)."""
    bg_frames = []
    for _ in range(frames):
        ok, frame = cap.read()
        if not ok:
            break
        frame = cv.flip(frame, 1)
        bg_frames.append(frame.astype(np.float32))
        cv.waitKey(10)
    if not bg_frames:
        return None
    return np.mean(bg_frames, axis=0).astype(np.uint8)

def main():
    cap = cv.VideoCapture(CAMERA_INDEX)
    if not cap.isOpened():
        raise SystemExit("❌ Camera not available")

    print("Warming up camera...")
    for _ in range(10):
        cap.read()

    print("Capturing background in 2 seconds. Step out of frame...")
    cv.waitKey(2000)
    background = capture_background(cap)
    if background is None:
        raise SystemExit("❌ Failed to capture background")

    color_ranges = get_color_ranges(CLOAK_COLOR)

    fourcc = cv.VideoWriter_fourcc(*"XVID")
    out, recording = None, False

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame = cv.flip(frame, 1)
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

        mask = build_mask(hsv, color_ranges)
        mask_inv = cv.bitwise_not(mask)

        cloak_area = cv.bitwise_and(background, background, mask=mask)
        non_cloak_area = cv.bitwise_and(frame, frame, mask=mask_inv)
        final = cv.addWeighted(cloak_area, 1, non_cloak_area, 1, 0)

        if SHOW_MASKS:
            cv.imshow("mask", mask)
            cv.imshow("mask_inv", mask_inv)

        cv.imshow("Invisible Cloak", final)

        if recording:
            if out is None:
                h, w = final.shape[:2]
                out = cv.VideoWriter("output.avi", fourcc, 30.0, (w, h))
            out.write(final)

        key = cv.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('b'):
            print("Re-capturing background...")
            cv.waitKey(500)
            background = capture_background(cap)
        elif key == ord('r'):
            recording = not recording
            print(f"Recording: {recording}")
            if not recording and out is not None:
                out.release()
                out = None
        elif key == ord('s'):
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            fname = f"frame_{ts}.png"
            cv.imwrite(fname, final)
            print(f"Snapshot saved: {fname}")

    cap.release()
    if out is not None:
        out.release()
    cv.destroyAllWindows()

# Run directly
main()


Warming up camera...
Capturing background in 2 seconds. Step out of frame...


In [ ]:
q

## 